# 检查shape以及label

In [ ]:
import os, shutil
from onekey_algo import get_param_in_cwd
import numpy as np
import nibabel as nib
import pandas as pd
from glob import glob

root = r'D:\20240416-GuanBo\T2'
for sample in glob(os.path.join(root, '*/*.nii.gz')):
    sname = os.path.basename(sample).replace('.nii.gz', '')
    sname = f"{int(sname):03d}"
    save2 = os.path.join(os.path.dirname(sample), f"{sname}.nii.gz")
    print(save2)
    shutil.move(sample, save2)

In [ ]:
import pandas as pd
import re

info = pd.read_csv(r'Z:/20240416-GuanBo/T2/linfo.csv')
info['病理诊断'] = info['病理诊断'].map(lambda x: re.sub(r'[（） ]', '', x))
info['Neurological'] = (info['病理诊断'].str.contains('神经侵犯+')).astype(int)
info['vascular'] = (info['病理诊断'].map(lambda x: 1 if '脉管侵犯+' in x or '脉管癌栓+' in x else 0)).astype(int)
info.to_csv(r'Z:/20240416-GuanBo/T2/info.csv', index=False, encoding='utf-8-sig')
info

In [ ]:
import os

a = set([f.replace('.npy', '.gz') for f in os.listdir(r'Z:\20240416-GuanBo\T2\roi_rad_features')])
b = set(os.listdir(r'Z:/20240416-GuanBo/T2/images/'))
b - a

# 划分数据

In [ ]:
import os
import re
import shutil
import pandas as pd
from onekey_algo.custom.components.comp2 import split_dataset4sol
from onekey_algo import get_param_in_cwd
from onekey_algo.custom.utils import print_join_info

mapping = pd.read_csv(r'Z:/20240416-GuanBo/T2/id_mapping.csv', 
                      converters={'pid': str, 'ID': lambda x: f"{int(x):03d}.nii.gz"})
data = pd.read_csv(r'Z:/20240416-GuanBo/T2/label-N.csv', dtype={'pid': str})
data = pd.merge(mapping, data, on='pid', how='left').drop_duplicates('ID')
samples = pd.read_csv('features/rad_features_habitat.csv')
data = pd.merge(samples['ID'], data, on='ID', how='left')[['ID', 'label']]
data = data.fillna(1)
data['label'] = data['label'].astype(int)
print_join_info(samples, data)
display(data)

rt = split_dataset4sol(data, data['label'], cv=False, n_trails=10, test_size=0.33, save_dir='.', shuffle=True)
# for idx, (train, val) in enumerate(rt):    
#     pd.merge(train, samples, on='ID', how='inner')[['fname', 'label']].to_csv(f'split_info/train-RND-{idx}.txt', 
#                                                                               sep='\t', header=None, index=None)
#     pd.merge(val, samples, on='ID', how='inner')[['fname', 'label']].to_csv(f'split_info/val-RND-{idx}.txt',
#                                                                             sep='\t', header=None, index=None)

In [ ]:
import pandas as pd

def split_g(x):
    try:
        return int(str(x).split("=")[-1])
    except:
        return None
c1 = pd.read_csv('Z:/20240416-GuanBo/T2/Clinical1.csv', dtype={"ID": str})
c1['Gleason'] = c1['Gleason'].map(lambda x: split_g(x))
c2 = pd.read_csv('Z:/20240416-GuanBo/T2/Clinical2.csv', dtype={"ID": str})
group = pd.read_csv('group.csv')
mapping = pd.read_csv(r'Z:/20240416-GuanBo/T2/id_mapping.csv', 
                      converters={'pid': str, 'ID': lambda x: f"{int(x):03d}.nii.gz"})
data = pd.merge(group, mapping, on='ID', how='left').drop_duplicates('ID')
c = pd.merge(c1, c2, on='ID', how='inner').drop_duplicates('ID')
c = pd.merge(data, c, right_on='ID', left_on='pid', how='left')
c.to_csv('Z:/20240416-GuanBo/T2/Clinical.csv', index=False, encoding='utf-8-sig')
c

In [ ]:
data